In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import keras
from IPython.display import SVG
from keras.optimizers import Adam, SGD
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

In [ ]:
DATAPATH = "data/"

In [ ]:
dataset = pd.read_csv(DATAPATH+"cleaned_data_train.csv", names="user_id,movie_id,prediction".split(','))
dataset['user_id'] = dataset['user_id']-1
dataset['movie_id'] = dataset['movie_id']-1
dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.1)

In [ ]:
n_users, n_movies = len(dataset.user_id.unique()), len(dataset.movie_id.unique())

In [ ]:
n_latent_factors_user = 5
n_latent_factors_movie = 8

movie_input = keras.layers.Input(shape=[1],name='Item')
movie_embedding = keras.layers.Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding')(movie_input)
movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)
movie_vec = keras.layers.Dropout(0.2)(movie_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_embedding = keras.layers.Embedding(n_users + 1, n_latent_factors_user,name='User-Embedding')(user_input)
user_vec = keras.layers.Flatten(name='FlattenUsers')(user_embedding)
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([movie_vec, user_vec])
concat_dropout = keras.layers.Dropout(0.2)(concat)

dense = keras.layers.Dense(300)(concat)
dense = keras.layers.Dropout(0.2)(dense)
dense_2 = keras.layers.Dense(150)(dense)
dense_2 = keras.layers.Dropout(0.2)(dense_2)
dense_3 = keras.layers.Dense(75)(dense_2)
dense_3 = keras.layers.Dropout(0.2)(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_3)
sgd = SGD(lr=0.05)
model = keras.Model([user_input, movie_input], result)
model.compile(optimizer=sgd,loss= 'mean_squared_error')



In [ ]:
model.summary()

# Put graphviz stuff heeeeere

In [ ]:
SVG(model_to_dot(model,  show_shapes=True, show_layer_names=True, rankdir='HB').create(prog='dot', format='svg'))


In [ ]:
history = model.fit([train.user_id, train.movie_id], train.prediction, epochs=50, verbose=1)

In [ ]:
y_hat = np.round(model.predict([test.user_id, test.movie_id]),0)
y_hat[y_hat < 1] = 1
y_hat[y_hat > 5] = 5
y_true = test.prediction

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_squared_error(y_true, y_hat)

In [ ]:
dataset_to_predict = pd.read_csv(DATAPATH+"cleaned_sample.csv", names="user_id,movie_id,prediction".split(','))
dataset_to_predict ['user_id'] = dataset_to_predict ['user_id']-1
dataset_to_predict ['movie_id'] = dataset_to_predict ['movie_id']-1
dataset_to_predict .head()

predictions = np.round(model.predict([dataset_to_predict.user_id, dataset_to_predict.movie_id]),0)
predictions[predictions < 1] = 1
predictions[predictions > 5] = 5

dataset_to_predict["prediction"] = predictions.astype(int)

def create_submission():
    f = open(DATAPATH+"submission4.csv", "w")
    f.write("Id,Prediction\n")
    for _,d in dataset_to_predict.iterrows():
        text = 'r'+str(d[0]+1)+'_c'+str(d[1]+1)+','+str(d[2])+'\n'
        f.write(text)
    f.close()
    
create_submission()